## SageMaker Model Registry 
haimtran 01/06/2023

In [ ]:
# !python -m pip install sagemaker 
# !python -m pip install boto3

In [ ]:
import json
import sagemaker 
import boto3

In [ ]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
sagemaker_client = boto3.client("sagemaker")

## Create a Model 

In [ ]:
from sagemaker.model import Model
from sagemaker.model_metrics import ModelMetrics
from sagemaker.image_uris import retrieve

In [ ]:
image_uri = retrieve(
    framework="pca",
    region=session._region_name, 
    instance_type="ml.m4.xlarge",
    version="0.23-1"
)

In [ ]:
image_uri

In [ ]:
model = Model(
    image_uri=image_uri,
    model_data="s3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz",
    role=role,
)

In [ ]:
# model.create(instance_type="ml.m4.xlarge")

## Create a Model Package Group 

In [ ]:
model_package_group_input_dict = {
    "ModelPackageGroupName": "ecg-model-group", 
    "ModelPackageGroupDescription": "develop model for ECG"
}

In [ ]:
# response = sagemaker_client.create_model_package_group(
#  **model_package_group_input_dict 
# )

In [ ]:
print(json.dumps(response, indent=4))

## Register a Model to a Model Group 

option 1. mode.create(), run model.register()

In [ ]:
# model.register(
#     model_package_group_name="ecg-model-group",
#     content_types=["text/csv"], 
#     inference_instances=["ml.m5.xlarge"], 
#     transform_instances=["ml.m5.xlarge"], 
#     response_types=["text/csv"], 
#     customer_metadata_properties={"ModelType": "PCA"}, 
#     # model_metrics=None,
#     approval_status="PendingManualApproval"
# )

option 2. using boto3 sagemaker client 

In [ ]:
# create_model_package_input_dict = {
#     "ModelPackageGroupName": "ecg-model-group",
#     "ModelPackageDescription": "", 
#     "ModelApprovalStatus" : "PendingManualApproval",
#     "InferenceSpecification": {
#         "Containers": [
#             {
#                 "Image": image_uri,
#                 "ModelDataUrl": "s3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz"
#             }
#         ],
#       "SupportedContentTypes": [ "text/csv" ],
#       "SupportedResponseMIMETypes": [ "text/csv" ],
#     }
# }

In [ ]:
# response = sagemaker_client.create_model_package(
#     **create_model_package_input_dict
# )
# print(json.dumps(response, indent=4))

## Describe Model Package 

In [ ]:
sagemaker_client.list_model_packages(
    ModelPackageGroupName="ecg-model-group"
)

In [ ]:
sagemaker_client.describe_model_package(
    ModelPackageName="arn:aws:sagemaker:ap-southeast-1:394599967678:model-package/ecg-model-group/1"
)

## Deploy a Model from Model Registry 

option 1. after model.create() run model.deploy()

option 2. using boto3 sagemaker client 

In [ ]:
endpoint_config_name = "PCAEndpointConfiguration"
endpoint_name = "PCAEndpointFromModelRegistry"
model_name = "pca-2023-06-01-07-52-59-642"

In [ ]:
endpoint_config = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            #  "ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ]
)

In [ ]:
response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
)

## Test the Endpoint 

In [ ]:
response = sagemaker_client.describe_endpoint(
    EndpointName=endpoint_name
)
print(json.dumps(response, indent=4, default=str))

In [ ]:
runtime = boto3.client("sagemaker-runtime")

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
     Body="1,2,3,4\n1,2,3,4\n2,3,4,5",
    ContentType="text/csv"
)
result = json.loads(response['Body'].read())
print(json.dumps(result, indent=4, default=str))